In [6]:
import pandas as pd
import numpy as np
import nltk
from cogroo4py.cogroo import Cogroo
# nltk.download('punkt')
import re
import language_tool_python as lt
import uol_redacoes_xml as uol

### Generate Datasets Possible Erros in Essays

In [29]:
tool = lt.LanguageTool('pt-BR')
cogroo = Cogroo()
essays = uol.load()

In [84]:
def generate_essay_dataframe(essays):
    data = []
    for i, ess in enumerate(essays):
        data.append({
            'essay': i,
            'final_score': ess.final_score,
            'text': ess.text,
            'comp1': ess.criteria_scores['Competência 1'],
            'comp2': ess.criteria_scores['Competência 2'],
            'comp3': ess.criteria_scores['Competência 3'],
            'comp4': ess.criteria_scores['Competência 4'],
            'comp5': ess.criteria_scores['Competência 5'],
            'title': ess.title,
            'url': ess.url,
            'comments': ess.comments
            
        })
    essay_df = pd.DataFrame(data)
    return essay_df

In [47]:
def split_sentences(text):
    regex =  r'(?<=[.!?])\s*(?=(?:[^"]*"[^"]*")*[^"]*$)'  # Expressão regular para separar as sentenças

    sentences = re.split(regex, text)

    return sentences

In [48]:
def get_essays_corrections_tool(essays, tool):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            matches = tool.check(sent)
            for mat in matches:
                data.append({
                    'essay': i,
                    'error': mat.ruleId,
                    'message': mat.message,
                    'category': mat.category,
                    'frase': sent
                })
    erros_df = pd.DataFrame(data)
    return erros_df

In [49]:
def get_essays_corrections_cogroo(essays, cogroo):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            try:
                doc = cogroo.grammar_check(sent)
                matches = doc.mistakes
                for mat in matches:
                    data.append({
                        'essay': i,
                        'error': mat.rule_id,
                        'message':  mat.short_msg,
                        'frase': sent
                    })
            except:
                print('Missed sentence', sent)
    erros_df = pd.DataFrame(data)
    return erros_df

In [51]:
tool_df = get_essays_corrections_tool(essays, tool)
tool_df.to_csv('../data/correction_language_tool_2.csv', sep='|', index=False)
tool_df.head()

,essay,error,message,category,frase
0,0,PT_WEASELWORD_REPLACE,‘bom senso’ é uma expressão ambígua e evasiva....,STYLE,"Quando um entra em contato com o outro, é prec..."
1,0,PT_WORDINESS_REPLACE,‘por meio de’ é uma expressão prolixa. É prefe...,REDUNDANCY,"Nessas épocas, as pessoas também sofriam grave..."
2,0,HUNSPELL_RULE,Encontrado possível erro de ortografia.,TYPOS,"Nessas épocas, as pessoas também sofriam grave..."
3,0,SPACE_BEFORE_PUNCTUATION,Remova o espaço antes deste sinal de pontuação.,TYPOGRAPHY,"Nessas épocas, as pessoas também sofriam grave..."
4,0,PT_COMPOUNDS_POST_REFORM,Esta palavra é hifenizada.,COMPOUNDING,"Dessa forma, o bem estar do ser humano deve es..."


In [52]:
cogroo_df = get_essays_corrections_cogroo(essays, cogroo)
cogroo_df.to_csv('../data/correction_cogroo_2.csv', sep='|', index=False)
cogroo_df.head()

TypeError: expected string or bytes-like object

In [85]:
essays_df = generate_essay_dataframe(essays)
essays_df.to_csv('../data/essays.csv', sep='|', index=False)
essays_df.head()

,essay,final_score,text,comp1,comp2,comp3,comp4,comp5,title,url,comments
0,0,5.0,"A liberdade de expressão, assim como o direito...",1.0,1.0,1.0,1.0,1.0,Conflito entre liberdade de expressão e intimi...,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nTexto regular. ...
1,1,6.5,Os direitos fundamentais contidos na Constitui...,1.0,2.0,1.5,1.0,1.0,Entre direitos individuais e escândalos políticos,https://educacao.uol.com.br/bancoderedacoes/re...,"<comments>## Comentário geral\nTexto regular, ..."
2,2,2.5,"Ao se falar em liberdade de expressão, é impos...",0.5,0.5,0.5,0.5,0.5,"O freio na liberdade, para uma maior liberdade",https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nAlém dos muitos...
3,3,2.5,Sabe-se que o cidadão tem direito à privacidad...,1.0,1.0,0.5,0.0,0.0,Privilégio à informação,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nO texto começa ...
4,4,5.0,A liberdade de expressão e o direito a vida pr...,1.0,1.0,1.0,1.0,1.0,Limites entre Direitos,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nTexto regular. ...
